In [1070]:
import pandas as pd


Chargement des données

In [1071]:
train = pd.read_csv("home-data-for-ml-course\\train.csv")
test = pd.read_csv("home-data-for-ml-course\\test.csv")
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()
print(train.shape)
print(test.shape)


(1460, 81)
(1459, 80)


In [1072]:
sale_price_df= train["SalePrice"]
train=train.drop(columns="SalePrice")
data= pd.concat([train,test],axis=0,ignore_index=True)
id_df= data["Id"]
data= data.drop(columns="Id")
print(data.shape)

(2919, 79)


Separation des colonnes selon leurs type

In [1073]:
data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]] = data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]].apply(pd.to_numeric, errors="coerce")

data_categorical_cols = [feature for feature in data.columns if data[feature].dtype == "object"] 

data_nominal_cols = ["MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", 
                    "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating",
                    "CentralAir", 'Electrical',"GarageType", "MiscFeature", "SaleType", "SaleCondition"]

data_ordinal_cols = [ 'LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                    'BsmtFinType2','HeatingQC','KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual',
                    'GarageCond','PavedDrive','PoolQC','Fence']

data_numerical_cols = [feature for feature in data.columns if feature not in data_categorical_cols]

data_discrete_numerical_cols = [ 'OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath',
  'FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','MoSold', "MSSubClass"] 

data_continuous_numerical_cols = ['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
                                  'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageYrBlt',
                                  'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',
                                  'MiscVal','YrSold','SalePrice']

Preprocessing

Missing values

In [1074]:
from sklearn.impute import SimpleImputer
print("numerical before: ",data[data_numerical_cols].isnull().sum().sum())
imputer = SimpleImputer(strategy='most_frequent')
data[data_numerical_cols]=pd.DataFrame(imputer.fit_transform(data[data_numerical_cols]),columns=data_numerical_cols)
print("numerical after: ",data[data_numerical_cols].isnull().sum().sum())


numerical before:  678
numerical after:  0


In [1075]:
print("categorical before: ",data[data_categorical_cols].isnull().sum().sum())
data[data_categorical_cols]=data[data_categorical_cols].fillna("NA")
print("categorical after: ",data[data_categorical_cols].isnull().sum().sum())

print(data.isnull().sum().sum(),"No null values left")

categorical before:  7548
categorical after:  0
0 No null values left


Encoding 

Ordinal features

In [1076]:
ordinal_mapping = {
    "LotShape": {"IR3": 1, "IR2": 2, "IR1": 3, "Reg": 4},
    "Utilities": {"ELO": 1, "NoSeWa": 2, "NoSewr": 3, "AllPub": 4},
    "LandSlope": {"Sev": 1, "Mod": 2, "Gtl": 3},
    "ExterQual": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "ExterCond": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "BsmtQual": {"NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "BsmtCond": {"NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "BsmtExposure": {"NA": 0, "No": 1, "Mn": 2, "Av": 3, "Gd": 4},
    "BsmtFinType1": {"NA": 0, "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6},
    "BsmtFinType2": {"NA": 0, "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6},
    "HeatingQC": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "KitchenQual": {"Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "Functional": {"Sal": 1, "Sev": 2, "Maj2": 3, "Maj1": 4, "Mod": 5, "Min2": 6, "Min1": 7, "Typ": 8},
    "FireplaceQu": {"NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "GarageFinish": {"NA": 0, "Unf": 1, "RFn": 2, "Fin": 3},
    "GarageQual": {"NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "GarageCond": {"NA": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    "PavedDrive": {"N": 1, "P": 2, "Y": 3},
    "PoolQC": {"NA": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4},
    "Fence": {"NA": 0, "MnWw": 1, "GdWo": 2, "MnPrv": 3, "GdPrv": 4}
}
ordinal_mapping = {col: ordinal_mapping[col] for col in data_ordinal_cols}
print('making sure the mapping contains all the ordinal features')
print(set(ordinal_mapping.keys())==set(data_ordinal_cols))

for col in ordinal_mapping.keys():
    if col in data.columns:
        data[col] = data[col].astype(str).str.strip()

making sure the mapping contains all the ordinal features
True


In [1077]:
from sklearn.preprocessing import OrdinalEncoder
categories = [list(ordinal_mapping[col].keys()) for col in data_ordinal_cols]
ordinal_encoder = OrdinalEncoder(categories=categories,handle_unknown="use_encoded_value", unknown_value=-1)
data[data_ordinal_cols] = ordinal_encoder.fit_transform(data[data_ordinal_cols])


Nominal Features

In [1078]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data[data_nominal_cols]= data[data_nominal_cols].apply(label_encoder.fit_transform)

Matrice de Correleation

In [1079]:
corr_threashold=0.75

In [1080]:
import numpy as np
corr_matrix= data[data_numerical_cols].corr()

high_corr_pairs = corr_matrix.abs() >= corr_threashold
np.fill_diagonal(high_corr_pairs.values, False)
high_corr_features = high_corr_pairs.stack()
high_corr_feature_names = high_corr_features[high_corr_features].index.tolist()
for i in range(0, len(high_corr_feature_names), 2):
    pair = high_corr_feature_names[i:i+2]
    print(pair)

[('TotalBsmtSF', '1stFlrSF'), ('1stFlrSF', 'TotalBsmtSF')]
[('GrLivArea', 'TotRmsAbvGrd'), ('TotRmsAbvGrd', 'GrLivArea')]
[('GarageCars', 'GarageArea'), ('GarageArea', 'GarageCars')]


In [1081]:
# dropping highly correlated features
cols_to_drop=['GarageYrBlt','TotalBsmtSF','GarageCars','TotRmsAbvGrd']
print("before: ",data.shape)
data.drop(columns=cols_to_drop,inplace=True)
print("after: ",data.shape)


before:  (2919, 79)
after:  (2919, 75)


Mutual Information

In [ ]:
mi_threashold=0.1

In [1083]:
from sklearn.feature_selection import mutual_info_regression
mi = mutual_info_regression(data.iloc[0:1460,:], sale_price_df)
mi_series = pd.Series(mi, index=data.columns)
mi_series = mi_series.sort_values(ascending=False)

cols_to_drop = mi_series[mi_series < mi_threashold].index.tolist()

print("Features with mutual information less than ",mi_threashold)
print(cols_to_drop)


Features with mutual information less than  0.2
['Foundation', 'HeatingQC', 'LotFrontage', 'Fireplaces', 'Exterior2nd', 'OpenPorchSF', 'LotArea', 'BsmtFinType1', 'MSZoning', 'BsmtFinSF1', 'Exterior1st', 'BsmtUnfSF', 'OverallCond', 'MasVnrType', 'LotShape', 'WoodDeckSF', 'BsmtExposure', 'HouseStyle', 'HalfBath', 'MasVnrArea', 'GarageCond', 'CentralAir', 'SaleType', 'GarageQual', 'BedroomAbvGr', 'Electrical', 'SaleCondition', 'BsmtCond', 'PavedDrive', 'ScreenPorch', 'BldgType', 'Fence', 'ExterCond', 'Alley', 'Functional', 'EnclosedPorch', 'BsmtFinType2', 'BsmtFullBath', 'LotConfig', 'KitchenAbvGr', 'Condition1', 'BsmtHalfBath', 'LandContour', 'RoofStyle', 'Heating', 'RoofMatl', 'LandSlope', 'Street', 'LowQualFinSF', 'PoolArea', 'BsmtFinSF2', '3SsnPorch', 'PoolQC', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'Condition2', 'Utilities']


In [1084]:
# dropping low mi features
data.drop(columns=cols_to_drop,inplace=True)
data.shape


(2919, 16)

Scalling Data

In [1085]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns=data.columns)
data.head()


,MSSubClass,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,ExterQual,BsmtQual,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,KitchenQual,FireplaceQu,GarageType,GarageFinish,GarageArea
0,0.067331,-1.207040,0.646183,1.046258,0.896833,1.039805,0.577094,-0.773861,1.207379,0.413547,0.781366,0.737021,-0.978830,-0.633105,0.316551,0.349364
1,-0.873616,1.983868,-0.063185,0.154764,-0.395604,-0.683756,0.577094,0.261075,-0.785025,-0.471891,0.781366,-0.766379,0.682014,-0.633105,0.316551,-0.058991
2,0.067331,-1.207040,0.646183,0.980221,0.848965,1.039805,0.577094,-0.610718,1.235375,0.563755,0.781366,0.737021,0.682014,-0.633105,0.316551,0.627787
3,0.302568,-1.039098,0.646183,-1.859351,-0.682812,-0.683756,-0.527521,-0.506205,0.978742,0.427382,-1.027363,0.737021,1.235629,1.419410,-0.798060,0.785561
4,0.067331,0.472385,1.355551,0.947203,0.753229,1.039805,0.577094,-0.037170,1.671651,1.378042,0.781366,0.737021,0.682014,-0.633105,0.316551,1.685798


In [1086]:
sale_price_df.to_csv("Y_train.csv",index=False)
data.to_csv("tayeb_data.csv",index=False)